In [7]:
import requests
import json
from collections import defaultdict

# GitHub repository information
owner = 'lucamenotti'
repo = '5v5s'
branch = 'main'  # Or any other branch you want to fetch from

# GitHub Personal Access Token
token = '' #add access token here

# GitHub API endpoint to list files in the repository
api_url = f'https://api.github.com/repos/{owner}/{repo}/contents'

# Alias mapping (key: alias, value: primary name)
player_aliases = {
    'EnderDragon7474': 'Ashalo1',
    'Ashalo1': 'Ashalo1',
    'RaVe B00ST': 'Bryant',
    'Coomer Hashira': 'Bryant',
    # Add more mappings as needed
}

def get_primary_name(name):
    """Returns the primary name for a given player name using the alias mapping."""
    return player_aliases.get(name, name)  # Return primary name if alias exists, otherwise the name itself

def fetch_files_from_github(api_url, path=''):
    """Fetches JSON files from the given GitHub repository path."""
    files = []
    headers = {'Authorization': f'token {token}'}  # Use token for authentication
    response = requests.get(f'{api_url}/{path}', headers=headers)
    if response.status_code == 200:
        for item in response.json():
            if item['type'] == 'file' and item['name'].endswith('.json'):
                files.append(item['download_url'])
            elif item['type'] == 'dir':
                files.extend(fetch_files_from_github(api_url, item['path']))
    else:
        print(f"Failed to fetch files: {response.status_code} - {response.text}")
    return files

def fetch_json_from_url(url):
    """Fetches JSON data from the given URL."""
    headers = {'Authorization': f'token {token}'}  # Use token for authentication
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch JSON from {url}: {response.status_code} - {response.text}")
    return None

# Fetch JSON files from the GitHub repository
json_files_urls = fetch_files_from_github(api_url)
#print("JSON Files URLs:", json_files_urls)

# Initialize player stats dictionary
player_stats = defaultdict(lambda: {
    'wins': 0,
    'losses': 0,
    'kills': 0,
    'deaths': 0,
    'matches': 0,
    'total_kills': 0,
    'games_played': 0
})

# Parse and extract data from each JSON file
for file_url in json_files_urls:
    match = fetch_json_from_url(file_url)
    if match:
        print(f"Processing file: {file_url}")  # Log file being processed
        if 'participants' not in match:
            print(f"No participants data in file: {file_url}")
            continue
        participants = match['participants']
        for player in participants:
            if 'NAME' not in player:
                print(f"No player name in file: {file_url}")
                continue
            player_name = get_primary_name(player['NAME'])  # Use primary name
            player_stats[player_name]['matches'] += 1
            player_stats[player_name]['kills'] += int(player.get('CHAMPIONS_KILLED', 0))
            player_stats[player_name]['deaths'] += int(player.get('NUM_DEATHS', 0))
            player_stats[player_name]['total_kills'] += int(player.get('CHAMPIONS_KILLED', 0))
            player_stats[player_name]['games_played'] += 1
            
            # Check win/loss
            if player.get('WIN') == 'Win':
                player_stats[player_name]['wins'] += 1
            else:
                player_stats[player_name]['losses'] += 1

if not player_stats:
    print("No player data found.")
else:
    # Compute win rate and average kills per match
    for player_name, stats in player_stats.items():
        stats['win_rate'] = stats['wins'] / stats['matches'] * 100
        stats['avg_kills_per_match'] = stats['kills'] / stats['matches']

    # Find player with the most deaths
    most_deaths_player = max(player_stats.items(), key=lambda x: x[1]['deaths'])
    most_kills_player = max(player_stats.items(), key=lambda x: x[1]['kills'])
    print(f"Player with the most deaths: {most_deaths_player[0]} with {most_deaths_player[1]['deaths']} deaths")
    print(f"Player with the most kills: {most_kills_player[0]} with {most_kills_player[1]['kills']} kills")
    
    # Print statistics for each player
    for player_name, stats in player_stats.items():
        print(f"Player: {player_name}")
        print(f"  Win Rate: {stats['win_rate']:.2f}%")
        print(f"  Average Kills per Match: {stats['avg_kills_per_match']:.2f}")
        print(f"  Total Deaths: {stats['deaths']}")
        print(f"  Total Kills: {stats['total_kills']}")
        print(f"  Games Played: {stats['games_played']}")
        print()


Processing file: https://raw.githubusercontent.com/lucamenotti/5v5s/main/4146140154.json?token=AWXZOWEZAHCBO24ZKPX2YTLGQNJOK
Processing file: https://raw.githubusercontent.com/lucamenotti/5v5s/main/4507145109.json?token=AWXZOWDVS2U3T5ONZ7IXPFDGQNJOK
Processing file: https://raw.githubusercontent.com/lucamenotti/5v5s/main/4523056888.json?token=AWXZOWB5NDCCZ7P73MPV57TGQNJOK
Processing file: https://raw.githubusercontent.com/lucamenotti/5v5s/main/4523110972.json?token=AWXZOWCFJZ3T2SH6KNFFGPLGQNJOK
Processing file: https://raw.githubusercontent.com/lucamenotti/5v5s/main/4523153945.json?token=AWXZOWCPSJTUK7NIUZXACGDGQNJOK
Processing file: https://raw.githubusercontent.com/lucamenotti/5v5s/main/4526839461.json?token=AWXZOWDUCQMIJITYGR5TI4LGQNJOK
Processing file: https://raw.githubusercontent.com/lucamenotti/5v5s/main/4526868306.json?token=AWXZOWE3RSCDWB6SYZNRHRLGQNJOK
Processing file: https://raw.githubusercontent.com/lucamenotti/5v5s/main/4576421436.json?token=AWXZOWHLG2KQB5I26LV5FQDGQNJOK
